In [ ]:
"""Libraries to install for the project"""

# pip install Cython python-dotenv
#pip install prophet
#pip install prophet
#pip install pystan==2.19.1.1
#pip install dotenv
#conda install -c conda-forge fbprophet
# conda install -c conda-forge prophet
# python -m pip install --upgrade pip
# conda install -c conda-forge prophet
# python -m pip install --upgrade pip
#from prophet import Prophet
#pip install pandas yfinance fbprophet plotly


In [3]:
import yfinance as yf
from prophet import Prophet
import pandas as pd
import plotly.graph_objs as go

def predict_stock_price_prophet(stock_ticker, end_date, duration_days, num_shares, investment_amount):
    # Calculate start date as 1825 days before the end_date
    start_date = end_date - pd.DateOffset(days=1825)
    
    # Download historical data
    data = yf.download(stock_ticker, start=start_date, end=end_date)
    
    # Prepare the data for Prophet
    df_prophet = pd.DataFrame(data['Close'].reset_index())
    df_prophet.columns = ['ds', 'y']
    
    # Initialize and fit the Prophet model
    model = Prophet(daily_seasonality=True)
    model.fit(df_prophet)
    
    # Make a future dataframe for predictions
    future = model.make_future_dataframe(periods=duration_days)
    
    # Predict future stock prices
    forecast = model.predict(future)
    
    # Find the predicted price for the end of the prediction period
    future_price = forecast['yhat'].iloc[-1]
    
    # Decision making (simplified, as trend analysis is more complex with Prophet)
    trend = 'higher' if future_price > df_prophet['y'].iloc[-1] else 'lower'
    action = 'buy' if trend == 'higher' else 'sell'
    optimal_volume = min(investment_amount / df_prophet['y'].iloc[-1], num_shares)
    investment_value = future_price * num_shares  # This is speculative
    
    # Visualization
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Predicted Price'))
    fig.add_trace(go.Scatter(x=df_prophet['ds'], y=df_prophet['y'], mode='lines', name='Actual Price'))
    fig.update_layout(title=f'Stock Prices and Predictions for {stock_ticker}',
                      xaxis_title='Date',
                      yaxis_title='Price')
    fig.show()
    
    return future_price, trend, action, optimal_volume, investment_value

# User input section
stock_ticker = input("Enter the stock ticker: ")
duration_days = int(input("Enter duration in time (forward in time in days): "))
end_date = pd.to_datetime(input("Enter today's date (YYYY-MM-DD): ")) + pd.DateOffset(days=duration_days)
num_shares = int(input("Enter the number of shares owned: "))
investment_amount = float(input("Enter how much you have to invest: "))

# Call the function
future_price, trend, action, optimal_volume, investment_value = predict_stock_price_prophet(stock_ticker, end_date, duration_days, num_shares, investment_amount)
print(f"Future predicted price after {duration_days} days: {future_price}")
print(f"The trend for the future price is {trend}.")
print(f"You should {action}.")
print(f"The optimal volume to {action} is {optimal_volume} shares.")
print(f"The speculative future value of your investment is: ${investment_value:.2f}.")
print("Please note, this is a prediction based on historical data and carries risk.")


[*********************100%%**********************]  1 of 1 completed
19:06:19 - cmdstanpy - INFO - Chain [1] start processing
19:06:20 - cmdstanpy - INFO - Chain [1] done processing
c:\ProgramData\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Future predicted price after 180 days: 13.703821051448855
The trend for the future price is lower.
You should sell.
The optimal volume to sell is 328.5151059314719 shares.
The speculative future value of your investment is: $13703.82.
Please note, this is a prediction based on historical data and carries risk.
